In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  


In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a model to get started

In [5]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to hear from you. How can I assist you today?


In [7]:
print(response)

ChatCompletion(id='chatcmpl-C6es3A7o3gnFjmB4CgGxHXirKhSdc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! Welcome! I'm glad you're here. How can I assist you today?", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1755703111, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=16, prompt_tokens=22, total_tokens=38, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


## Onwards with our first project

In [30]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 25.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [selenium]5/6 [selenium]


In [31]:
!pip install webdriver-manager

In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import os 

class WebsiteSel:
    def __init__(self, url, driver_path=None, wait_time=3):
        self.url = url
        self.wait_time = wait_time

        # Headless Chrome settings
        options = Options()
        # options.add_argument("--headless")  
        # Headless mode runs the browser in the background (invisible).
        # However, some websites (like openai.com) block headless browsers.
        # So if this line is active, the page may not load correctly and you may not get the full content.
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        options.add_argument("--window-size=1920x1080")

        # Driver path
        if driver_path:
            service = Service(executable_path=driver_path)
        else:
            service = Service() 

        # Start browser
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # Wait for the loading page
        time.sleep(self.wait_time)

        # Take page source
        html = driver.page_source
        driver.quit()

        # Analysis with BeautifulSoup 
        soup = BeautifulSoup(html, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

      
        # # Remove irrelevant tags: scripts, styles, images, inputs, navs, footers, headers, and asides
        for irrelevant in soup.body(["script", "style", "img", "input", "nav", "footer", "header", "aside"]):
            irrelevant.decompose()
        
        # Remove common ad containers (classes and IDs often used for ads/boilerplate)
        for ad in soup.body.find_all(attrs={
            "class": ["ad", "ads", "advertisement", "sponsored", "promo", "banner", "cookie-consent"],
            "id": ["ad", "ads", "advertisement", "sponsored", "promo", "banner", "cookie-consent"]
        }):
            ad.decompose()
        
        # Remove repetitive boilerplate sections (menus, sidebars, etc.)
        for boilerplate in soup.body.find_all(attrs={
            "class": ["sidebar", "menu", "navigation", "breadcrumb", "footer-links"]
        }):
            boilerplate.decompose()
        
        # Now extract cleaned text
        self.text = soup.body.get_text(separator="\n", strip=True)       

In [71]:
site = WebsiteSel("https://nykaa.com", driver_path="/Users/rahulrajrana/Downloads/chromedriver-mac-arm64/chromedriver")
print("Title:", site.title)
print("\nFirst 500 character:\n", site.text[:500])

Title: Buy Cosmetics Products & Beauty Products Online in India at Best Price | Nykaa

First 500 character:
 Up to 15% off
Free Gifts On ₹2500+
Flat 15% Off
On M.A.C Foundation
Up To 20% Off
On Kay Bestsellers!
Upto 20% Off
On Bestseller!
New Launch
Bubble has landed
Min 20% Off
On Entire Range!
Flat 50% Off
*On Entire Range
Up To 50% Off
On Bestsellers
Ends In:
04h 54m 16s
Only at ₹650/-
10% Off on 1200+
Flat 10% Off On Lips
On ₹6000: 3 Free Gifts
Up To 20% Off
On Entire Brand!
Up To 30% Off
On Trending Makeup!
With 7 Natural Oils
For All Hair Types!
AHA 30% + BHA 2%
For clear pores & even tone
Gifts 


In [47]:
# Define our system prompt"

system_prompt = "You are a helpful shopping assistant. \
Your job is to read product webpage content and explain it to a shopper in clear, simple language. \
Always provide a decision guide with key features, pros and cons, who the product is best for, and a short verdict. \
Keep the tone neutral, practical, and easy to scan.\
Respond in markdown."

In [68]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += f"""
Read the following product webpage content and generate a shopping decision guide. 

Use this structure:

Product: <name/category>  
Key Features:  
- ...  
Pros:  
- ...  
Cons:  
- ...  
Best For:  
- ...  
Verdict:  
- ...  """
    user_prompt += website.text
    return user_prompt

In [69]:
print(user_prompt_for(site))

You are looking at a website titled No title found
Read the following product webpage content and generate a shopping decision guide. 

Use this structure:

Product: <name/category>  
Key Features:  
- ...  
Pros:  
- ...  
Cons:  
- ...  
Best For:  
- ...  
Verdict:  
- ...  


## And now let's build useful messages for GPT-4o-mini, using a function

In [53]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [72]:
# Try this out, and then try for a few more websites

messages_for(site)

[{'role': 'system',
  'content': 'You are a helpful shopping assistant. Your job is to read product webpage content and explain it to a shopper in clear, simple language. Always provide a decision guide with key features, pros and cons, who the product is best for, and a short verdict. Keep the tone neutral, practical, and easy to scan.Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Buy Cosmetics Products & Beauty Products Online in India at Best Price | Nykaa\nRead the following product webpage content and generate a shopping decision guide. \n\nUse this structure:\n\nProduct: <name/category>  \nKey Features:  \n- ...  \nPros:  \n- ...  \nCons:  \n- ...  \nBest For:  \n- ...  \nVerdict:  \n- ...  Up to 15% off\nFree Gifts On ₹2500+\nFlat 15% Off\nOn M.A.C Foundation\nUp To 20% Off\nOn Kay Bestsellers!\nUpto 20% Off\nOn Bestseller!\nNew Launch\nBubble has landed\nMin 20% Off\nOn Entire Range!\nFlat 50% Off\n*On Entire Range\nUp To 50% Off\nOn

## Time to bring it together - the API for OpenAI!

In [73]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = WebsiteSel(url,driver_path="/Users/rahulrajrana/Downloads/chromedriver-mac-arm64/chromedriver")
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [62]:
summarize("https://nykaa.com")

'## Product: Cosmetics and Beauty Products\n\n### Key Features:\n- Wide range of products available from various brands.\n- Discounts up to 50% on selected items, including bestsellers.\n- Offers free gifts on purchases over ₹2500 or ₹3500.\n- New launches available, ensuring access to the latest trends.\n- Special deals including up to 30% off on trending makeup and flat rates like 10% off on lips.\n- Product range includes items suitable for all hair types with nourishing ingredients.\n\n### Pros:\n- Competitive pricing with frequent sales and discounts.\n- Promotions that include attractive free gifts, enhancing the shopping experience.\n- A diverse selection of beauty products catering to different needs and preferences.\n- Easy access to both bestselling and new arrivals in the beauty industry.\n\n### Cons:\n- Discounts may apply only for a limited time, creating urgency for buyers.\n- Minimum purchase amounts required to obtain free gifts may deter some shoppers.\n- Potential for

In [74]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [75]:
display_summary("https://nykaa.com/")

# Shopping Decision Guide

**Product:** Cosmetics Products & Beauty Products

**Key Features:**
- Wide range of products available, including makeup, skincare, and haircare.
- Special promotions like up to 50% off on bestsellers and flat discounts on specific brands.
- Offers free gifts for purchases over certain amounts (₹2500 and ₹6000).
- Trending makeup at discounted rates (up to 30% off).
- Products are suitable for various needs (e.g., for all hair types, pore clearing).

**Pros:**
- Competitive prices and frequent sales give good value.
- Exclusive brands and new launches available for trendy options.
- Offers promotions and bundles, which can enhance the shopping experience.
- Free shipping and gifts on larger purchases incentivize buying more.

**Cons:**
- Discounts are often time-limited (e.g., ending in hours), requiring quick decisions.
- Stock may vary, and popular items can sell out quickly.
- The sheer variety might make it overwhelming for some shoppers.

**Best For:**
- Beauty enthusiasts looking for a diverse selection of cosmetics and skincare products.
- Shoppers who enjoy finding good deals and discounts on popular brands.
- Individuals seeking gifts or special offers for themselves or others.

**Verdict:**
- Nykaa offers an extensive range of beauty products at competitive prices with excellent promotional offers. It's a great option for anyone interested in cosmetics, skincare, and haircare, particularly those who are looking to save money through discounts and free gifts. However, it's advisable to act quickly on deals to avoid missing out.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. 

But many websites will work just fine!

In [77]:
display_summary("https://www.amazon.in/boAt-Rockerz-480-Bluetooth-Headphones/dp/B0DGTSRX3R/ref=sr_1_4?_encoding=UTF8&content-id=amzn1.sym.f5d6180d-d975-4dbf-90a0-e12c627a7820&dib=eyJ2IjoiMSJ9.FxruVmk1CFBRRSpxWp9ryFm3c3a9zDqlDwLf2RM0QxzS6xvhWPAxoVhFYQATslvl1W57IvP9TSf0x4rdC52Zaz1adrYJHdqWIvgc-tn8_kU7EB1-g9Adl9b2f323kEr5E2sza12qpcf6jOfZ2PCP8Nt1mUYGtZJxqbBXXMO3_nMSM9CY9Xe6Rl-ewCtVm2rpY261U5Kb5Z6CXgEfWV56d6JHtBxrssqf-Z3j08Bj9g_qoKFft1QCamuM1XGGt_CKxBF1Q3bB9_4E4cvVWkZfPNEuX7JQ5fMYTP3t7iCTIP8.yuFJrOJ_hYPFvXNt-4RpdHKEt7pwP-6fS4kox5j9dmo&dib_tag=se&pd_rd_r=403a0988-23a6-4502-b268-73925227a2f5&pd_rd_w=hS00c&pd_rd_wg=CzLCv&qid=1755801463&sr=8-4&th=1")

# Shopping Decision Guide: boAt Rockerz 480 Bluetooth Headphones

**Product:** boAt Rockerz 480 Bluetooth Over Ear Headphones

### Key Features:
- **RGB LEDs with 6 Modes:** Customizable LED lights to enhance your audio experience.
- **BEAST Mode:** Reduces latency to 40 ms for gaming, ensuring a smooth experience.
- **60 Hours Playback:** Long battery life, allowing for extended use without frequent charging.
- **40 mm Drivers:** Delivers a bass-heavy sound for rich audio quality.
- **ENx Technology:** Enhances call clarity by minimizing background noise.
- **Dual Pairing:** Connects to two devices simultaneously, ideal for multitasking.
- **Bluetooth v5.3:** Ensures a stable and efficient wireless connection.
- **Adaptive Fit:** Comfortable design for extended wear.
- **Easy Access Controls:** Manage calls and audio quickly and easily.
- **Voice Assistant Compatibility:** Supports Google Assistant and Siri for hands-free operation.

### Pros:
- Impressive battery life of up to 60 hours.
- Good sound quality with a focus on bass.
- Stylish design with customizable LED lights.
- Comfortable for prolonged use due to adaptive fit.
- Fast and stable Bluetooth connection.

### Cons:
- Build quality may feel a bit plastic-y to some users.
- Background noise may occasionally interfere with call clarity, despite ENx technology.
- A few users felt sound quality could improve with an equalizer.

### Best For:
- Gamers seeking low-latency audio for enhanced gameplay.
- Music lovers looking for budget-friendly headphones with good sound quality.
- Anyone needing comfortable headphones for long listening sessions or calls.
- Those wanting headphones that can connect to multiple devices simultaneously.

### Verdict:
The boAt Rockerz 480 headphones offer a compelling mix of excellent battery life, vibrant sound quality, and usability features like dual pairing and voice assistant compatibility. While they may have some minor drawbacks in build quality and sound tuning for specific genres, their overall performance and comfort make them a great choice for gamers, music enthusiasts, or anyone needing reliable wireless headphones on a budget. 

**Overall Rating:** 4.1 out of 5 stars from over 2,100 ratings.

# Even converts Hindi News to it's English Equivalent Summary

In [27]:
display_summary("https://bhaskar.com")

# Summary of Dainik Bhaskar Website

Dainik Bhaskar is a comprehensive Hindi news website providing the latest updates on various topics, including politics, sports, entertainment, and lifestyle. The site offers real-time news coverage, videos, and special features on trending issues.

## Key Highlights:

- **Political News**: The site discusses significant political events like recent bills introduced in the Lok Sabha, including a bill that could lead to the removal of PM or CM upon arrest. There were also incidents of opposition protests where paper balls were thrown in Parliament.

- **Crime and Legal Matters**: Reports cover alarming incidents, such as an attack on Delhi's Chief Minister and issues surrounding migrant workers in Delhi-NCR. The aftermath of natural disasters, such as fatalities caused by cloudbursts in Kishtwar, is also highlighted.

- **Sports Updates**: Cricket features prominently, including discussions about India's team composition for the Asia Cup, and controversies surrounding the rankings of players like Rohit Sharma and Virat Kohli.

- **Business News**: Developments in the business world include the potential banning of online gaming sponsors and updates related to job recruitment in banks.

- **Lifestyle Articles**: Content spans lifestyle tips and relationship advice, focusing on the well-being of readers.

- **Trending Topics**: The site reports on various trending topics like the ongoing Russia-Ukraine war, the political landscape in Maharashtra, and unique stories captivating audience interest.

Overall, Dainik Bhaskar serves as a key source of information for Hindi-speaking audiences seeking diverse news coverage from local to international events.